In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append("./CLIP")
import clip
import os
import torch
from PIL import Image
from IPython.display import Image as ImageDisplay
from utils import load_pickle
import faiss_utils
from faiss_utils import KNearestFaiss
import numpy as np
from matplotlib.image import imread
from matplotlib.pyplot import figure, imshow, axis
from jupyter_utils import plot_time_jupyter, plot_scores_jupyter

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:


def img2array(im):
    if im.mode != 'RGB':
        im = im.convert(mode='RGB')
    return np.fromstring(im.tostring(), dtype='uint8').reshape((im.size[1], im.size[0], 3))

def display_top_query_from_meta(metadata_list, fill=True, score_list=[]):
    list_of_files = [meta.get_path() for meta in metadata_list]
    show_images(list_of_files, fill=fill, score_list=score_list)
#     for meta in metadata_list:
#         plt.figure()
#         plt.imshow(img2array(Image.open(meta.get_path())))

        

def show_images(list_of_files, fill=True, score_list=[]):
    block_size = int(len(list_of_files)/20)
    size = (20+20*block_size,20+90*block_size)
    print(size)
    fig = figure(figsize=size, tight_layout=True)
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(number_of_files,3,i+1)
        image = imread(list_of_files[i])
#         imshow(image,cmap='Greys_r')
        if fill:
            imshow(image,aspect="auto")
        else:
            imshow(image)
        if len(score_list) > 0:
            title=score_list[i]
            plt.title(title, fontsize=30); 
        axis('off')


def display_query_dict_random(query_dict, class_start=0, class_end=9, fill=False):
    query_names = list(query_dict.keys())[class_start:class_end+1]
    image_dict = {}
    rank_list=list(range(0, 999, 40))
    for query_name in query_names:
        list_of_files, list_of_scores, rank_list = select_from_meta(query_dict[query_name]['metadata'], query_dict[query_name]['D'], rank_list=rank_list)
        image_dict[query_name] = list_of_files, list_of_scores
    show_image_dict(image_dict, fill=fill)

def display_query_dict(query_dict, class_start=0, class_end=9, fill=False):
    query_names = list(query_dict.keys())[class_start:class_end+1]
    image_dict = {}
    for query_name in query_names:
        list_of_files, list_of_scores, rank_list = select_from_meta(query_dict[query_name]['metadata'], query_dict[query_name]['D'])
        image_dict[query_name] = list_of_files, list_of_scores
    show_image_dict(image_dict, fill=fill)
        
    
def select_from_meta(metadata_list, score_list, rank_list=[0, 199, 399, 599, 799, 999]):
    list_of_files = [metadata_list[rank].get_path() for rank in rank_list]
    list_of_scores = [score_list[rank] for rank in rank_list]
    return list_of_files, list_of_scores, rank_list
#     for meta in metadata_list:
#         plt.figure()
#         plt.imshow(img2array(Image.open(meta.get_path())))
 

def show_image_dict(image_dict, fill=True):
    block_size = 2
    size = (20+20*block_size,20+90*block_size)
    size = (20+20*block_size,20+420*block_size)
    print(size)
    fig = figure(figsize=size, tight_layout=True)
    number_of_queries = len(image_dict.keys())
    list_of_files = []
    list_of_scores = []
    list_of_queries = []
    for query in image_dict.keys():
        files, scores= image_dict[query]
        list_of_files += files
        list_of_queries += [query for _ in files]
        list_of_scores += scores
        length_per_query = len(files)
    number_of_files = len(list_of_files)
    
    for i in range(number_of_files):
        a=fig.add_subplot(number_of_files,int(length_per_query/5),i+1)
        if i % length_per_query == 0:
            a.set_ylabel(list_of_queries[i],  fontsize=28)

        image = imread(list_of_files[i])
#         imshow(image,cmap='Greys_r')
        if fill:
            imshow(image,aspect="auto")
        else:
            imshow(image)
        if len(list_of_scores) > 0:
            title=list_of_scores[i]
            plt.title(title, fontsize=30); 
        axis('on')

In [3]:
main_folder = "/scratch/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/"
# main_folder = "/compute/autobot-0-19/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/"
# label_set = "vehicle_7"
label_set = "tech_7"
# label_set = "cifar10"
# detail_str = "size_1000_doq_True"
# detail_str = "size_1000_doq_False"
detail_str = "query_photo_size_1000_nnsize_10000_maxscore"
# detail_str = "query_none_size_1000_nnsize_10000_maxscore"

save_path = os.path.join(main_folder, label_set, detail_str, "query_dict.pickle")
print(save_path)
assert os.path.exists(save_path)
query_dict  = load_pickle(save_path)
# display_query_dict(query_dict)
display_query_dict_random(query_dict)

/scratch/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/tech_7/query_photo_size_1000_nnsize_10000_maxscore/query_dict.pickle


AssertionError: 

In [4]:
main_folder = "/scratch/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/"
# main_folder = "/compute/autobot-0-19/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/"
# label_set = "vehicle_7"
# label_set = "tech_7"
# label_set = "sports_30"
label_set = "fashion_25"
# label_set = "cifar10"
# detail_str = "size_1000_doq_True"
# detail_str = "size_1000_doq_False"
detail_str = "query_photo_size_1000_nnsize_10000_maxscore"
# detail_str = "query_none_size_1000_nnsize_10000_maxscore"

save_path = os.path.join(main_folder, label_set, detail_str, "query_dict.pickle")
print(save_path)
assert os.path.exists(save_path)
query_dict  = load_pickle(save_path)
# display_query_dict(query_dict)
display_query_dict_random(query_dict)

/scratch/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset/fashion_25/query_photo_size_1000_nnsize_10000_maxscore/query_dict.pickle


AssertionError: 

In [24]:
main_folder = "/scratch/zhiqiu/yfcc100m_all/clip_dataset/images_minbyte_10_valid_uploaded_date_jan_31/clip_dataset_bucketed/"
label_set = "vehicle_7"
# label_set = "tech_7"
# label_set = "sports_30"
label_set = "cifar100"
label_set = "imagenet1K"
detail_str = "query_photo_size_1000_bucket_3"

num_of_buckets = 3


def cosine(f_1, f_2):
    return 1 - f_1.dot(f_2)

def norm_cosine(f_1, f_2):
    f_1 = f_1/np.linalg.norm(f_1)
    f_2 = f_2/np.linalg.norm(f_2)
    return 1 - f_1.dot(f_2)

def eu(f_1, f_2):
    return np.linalg.norm(f_1 - f_2)

DISTANCES = {
    'cosine' : cosine,
    'norm_cosine' : norm_cosine,
    'eu' : eu
}


def get_feature_mean_list(query, query_dict):
    feature_mean_list = []
    for b_idx in range(num_of_buckets):
#         import pdb; pdb.set_trace()
        feature_mean_list.append(query_dict[b_idx][query]['features_mean'])
    return feature_mean_list
    
def gather_distances(query, query_dict):
    feature_mean_list = get_feature_mean_list(query, query_dict)
    distances = {}
    for idx, f in enumerate(feature_mean_list):
        if idx == len(feature_mean_list) - 1:
            break
        for second_idx in range(idx+1, len(feature_mean_list)):
            f_2 = feature_mean_list[second_idx]
            distance_all = {}
            for distance_type in DISTANCES:
                distance_all[distance_type] = DISTANCES[distance_type](f, f_2)
            distances[(idx, second_idx)] = distance_all
    print(query)
    ds = {'cosine' : [], 'norm_cosine' : [], 'eu' : []}
    for (idx, idx_2) in distances:
        d = distances[(idx, idx_2)]
        for distance_type in DISTANCES:
            ds[distance_type].append(d[distance_type])
        print(f"   {idx}-to-{idx_2}: Cosine({d['cosine']:.5f}) NormCosine({d['norm_cosine']:.5f}) EU({d['eu']:.5f})")

    cos_distance_mean = sum(ds['cosine'])/len(ds['cosine'])
    norm_cos_distance_mean = sum(ds['norm_cosine'])/len(ds['norm_cosine'])
    eu_distance_mean = sum(ds['eu'])/len(ds['eu'])
    cos_distance_first_last = distances[(0, num_of_buckets-1)]['cosine']
    norm_cos_distance_first_last = distances[(0, num_of_buckets-1)]['norm_cosine']
    eu_distance_first_last = distances[(0, num_of_buckets-1)]['eu']
    print(f"Average: Cos({cos_distance_mean:.5f}), NormCos({norm_cos_distance_mean:.5f}), EU({eu_distance_mean:.5f}) for {query}")
    print(f"First Last: Cos({cos_distance_first_last:.5f}), NormCos({norm_cos_distance_first_last:.5f}), EU({eu_distance_first_last:.5f}) for {query}")
    return {
        'avg': {
            "cosine" : cos_distance_mean,
            'eu' : eu_distance_mean,
            'norm_cosine' : norm_cos_distance_mean,
        },
        'first_last' : {
            "cosine" : cos_distance_first_last,
            'eu' : eu_distance_first_last,
            'norm_cosine' : norm_cos_distance_first_last,
        }
    }

def print_all(distances_dict, key='avg', distance_type='cosine'):
    distances_dict_summary = []
    for query in distances_dict:
        # ds = []
        # for item in distances_dict[query]:
        #     ds.append(distances_dict[query][item]/len(list(distances_dict[query].keys())))
        distances_dict_summary.append((distances_dict[query][key][distance_type], query))
#     reverse = distance_type == 'eu'
    distances_dict_summary = sorted(distances_dict_summary, key=lambda x : x[0], reverse=True)
    print(f"{key}:{distance_type} for {label_set}. Top are:")
    for i in range(len(distances_dict_summary)):
        d, q = distances_dict_summary[i]
        print(f"{q:40s}: {d:.5f}")
#     print(f"{key}:{distance_type} for {label_set}. Bottom are:")
#     for i in range(-1, -6, -1):
#         d, q = distances_dict_summary[i]
#         print(f"{q}: {d}")

def print_query_dict(query_dict,  distance_type='norm_cosine'):
    distances_dict = {}
#     first_key = list(query_dict[0].keys())[0]
    for query in query_dict[0]:
        query_distances = gather_distances(query, query_dict)
        distances_dict[query] = query_distances

#     print_all(distances_dict, key='avg', distance_type=distance_type)
    print_all(distances_dict, key='first_last', distance_type=distance_type)
            

In [25]:

save_path = os.path.join(main_folder, label_set, detail_str, "distances.pickle")
save_path_0 = os.path.join(main_folder, label_set, detail_str, "0", "query_dict.pickle")
save_path_1 = os.path.join(main_folder, label_set, detail_str, "1", "query_dict.pickle")
save_path_2 = os.path.join(main_folder, label_set, detail_str, "2", "query_dict.pickle")
assert os.path.exists(save_path)
assert os.path.exists(save_path_0)
assert os.path.exists(save_path_1)
assert os.path.exists(save_path_2)
distance_dict  = load_pickle(save_path)
query_dict_0  = load_pickle(save_path_0)
query_dict_1  = load_pickle(save_path_1)
query_dict_2  = load_pickle(save_path_2)

# distance_dict.keys()
# print(query_dict_0[0]['A photo of a cat'].keys())
# text_feature = query_dict_0[0]['A photo of a cat']['text_feature']
# feature_0 = query_dict_0[0]['A photo of a cat']['features'][0]
# feature_mean = query_dict_0[0]['A photo of a cat']['features_mean']
# print(f"Text dot 0: {text_feature.dot(feature_0)}")
# print(f"Text dot mean: {text_feature.dot(feature_mean)}")
# print(f"mean dot 0: {feature_mean.reshape(1,-1).dot(feature_0)}")
# print(f"mean dot mean: {feature_mean.reshape(1,-1).dot(feature_mean)}")
# print(f"0 dot 0: {feature_0.reshape(1,-1).dot(feature_0)}")
# print(feature_mean.shape)
# print(text_feature.shape)
# print(feature_0.shape)

# display_query_dict(query_dict_0[0])
print(query_dict_2.keys())
print_query_dict(query_dict_2, distance_type='norm_cosine')
# print_query_dict(query_dict_2, distance_type='eu')

dict_keys([0, 1, 2])
A photo of a tench
   0-to-1: Cosine(0.34288) NormCosine(0.00284) EU(0.06341)
   0-to-2: Cosine(0.34314) NormCosine(0.00279) EU(0.06299)
   1-to-2: Cosine(0.35540) NormCosine(0.00089) EU(0.03395)
Average: Cos(0.34714), NormCos(0.00217), EU(0.05345) for A photo of a tench
First Last: Cos(0.34314), NormCos(0.00279), EU(0.06299) for A photo of a tench
A photo of a goldfish
   0-to-1: Cosine(0.34787) NormCosine(0.00149) EU(0.04536)
   0-to-2: Cosine(0.35124) NormCosine(0.00233) EU(0.05683)
   1-to-2: Cosine(0.35903) NormCosine(0.00105) EU(0.03695)
Average: Cos(0.35272), NormCos(0.00162), EU(0.04638) for A photo of a goldfish
First Last: Cos(0.35124), NormCos(0.00233), EU(0.05683) for A photo of a goldfish
A photo of a great white shark
   0-to-1: Cosine(0.31322) NormCosine(0.00250) EU(0.06118)
   0-to-2: Cosine(0.31152) NormCosine(0.00180) EU(0.05228)
   1-to-2: Cosine(0.32484) NormCosine(0.00063) EU(0.02922)
Average: Cos(0.31653), NormCos(0.00165), EU(0.04756) for A p

A photo of a Irish Terrier
   0-to-1: Cosine(0.36525) NormCosine(0.00231) EU(0.05526)
   0-to-2: Cosine(0.37435) NormCosine(0.00461) EU(0.07870)
   1-to-2: Cosine(0.38027) NormCosine(0.00105) EU(0.03737)
Average: Cos(0.37329), NormCos(0.00266), EU(0.05711) for A photo of a Irish Terrier
First Last: Cos(0.37435), NormCos(0.00461), EU(0.07870) for A photo of a Irish Terrier
A photo of a Norfolk Terrier
   0-to-1: Cosine(0.32481) NormCosine(0.00330) EU(0.06950)
   0-to-2: Cosine(0.33109) NormCosine(0.00404) EU(0.07802)
   1-to-2: Cosine(0.34399) NormCosine(0.00072) EU(0.03153)
Average: Cos(0.33329), NormCos(0.00269), EU(0.05968) for A photo of a Norfolk Terrier
First Last: Cos(0.33109), NormCos(0.00404), EU(0.07802) for A photo of a Norfolk Terrier
A photo of a Norwich Terrier
   0-to-1: Cosine(0.35629) NormCosine(0.00371) EU(0.07112)
   0-to-2: Cosine(0.36801) NormCosine(0.00487) EU(0.08413)
   1-to-2: Cosine(0.37836) NormCosine(0.00102) EU(0.03819)
Average: Cos(0.36755), NormCos(0.00320

A photo of a bobsleigh
   0-to-1: Cosine(0.47592) NormCosine(0.00161) EU(0.04201)
   0-to-2: Cosine(0.47839) NormCosine(0.00300) EU(0.05713)
   1-to-2: Cosine(0.48378) NormCosine(0.00140) EU(0.03809)
Average: Cos(0.47936), NormCos(0.00200), EU(0.04575) for A photo of a bobsleigh
First Last: Cos(0.47839), NormCos(0.00300), EU(0.05713) for A photo of a bobsleigh
A photo of a bolo tie
   0-to-1: Cosine(0.44944) NormCosine(0.00284) EU(0.05640)
   0-to-2: Cosine(0.45178) NormCosine(0.00409) EU(0.06774)
   1-to-2: Cosine(0.45521) NormCosine(0.00119) EU(0.03609)
Average: Cos(0.45215), NormCos(0.00271), EU(0.05341) for A photo of a bolo tie
First Last: Cos(0.45178), NormCos(0.00409), EU(0.06774) for A photo of a bolo tie
A photo of a poke bonnet
   0-to-1: Cosine(0.44869) NormCosine(0.00323) EU(0.06014)
   0-to-2: Cosine(0.45590) NormCosine(0.00636) EU(0.08457)
   1-to-2: Cosine(0.45804) NormCosine(0.00186) EU(0.04553)
Average: Cos(0.45421), NormCos(0.00382), EU(0.06341) for A photo of a poke 

A photo of a tights
   0-to-1: Cosine(0.36732) NormCosine(0.00199) EU(0.05347)
   0-to-2: Cosine(0.37425) NormCosine(0.00327) EU(0.06906)
   1-to-2: Cosine(0.38702) NormCosine(0.00107) EU(0.03702)
Average: Cos(0.37620), NormCos(0.00211), EU(0.05318) for A photo of a tights
First Last: Cos(0.37425), NormCos(0.00327), EU(0.06906) for A photo of a tights
A photo of a tank suit
   0-to-1: Cosine(0.49057) NormCosine(0.00600) EU(0.08035)
   0-to-2: Cosine(0.49704) NormCosine(0.01501) EU(0.12543)
   1-to-2: Cosine(0.50358) NormCosine(0.00384) EU(0.06196)
Average: Cos(0.49707), NormCos(0.00829), EU(0.08924) for A photo of a tank suit
First Last: Cos(0.49704), NormCos(0.01501), EU(0.12543) for A photo of a tank suit
A photo of a manhole cover
   0-to-1: Cosine(0.37663) NormCosine(0.00131) EU(0.04146)
   0-to-2: Cosine(0.38423) NormCosine(0.00295) EU(0.06271)
   1-to-2: Cosine(0.39016) NormCosine(0.00148) EU(0.04327)
Average: Cos(0.38367), NormCos(0.00191), EU(0.04915) for A photo of a manhole c

A photo of a toy store
   0-to-1: Cosine(0.45312) NormCosine(0.00286) EU(0.05617)
   0-to-2: Cosine(0.45761) NormCosine(0.00330) EU(0.06076)
   1-to-2: Cosine(0.46018) NormCosine(0.00268) EU(0.05412)
Average: Cos(0.45697), NormCos(0.00295), EU(0.05702) for A photo of a toy store
First Last: Cos(0.45761), NormCos(0.00330), EU(0.06076) for A photo of a toy store
A photo of a tractor
   0-to-1: Cosine(0.41962) NormCosine(0.00262) EU(0.05527)
   0-to-2: Cosine(0.42225) NormCosine(0.00710) EU(0.09094)
   1-to-2: Cosine(0.42083) NormCosine(0.00209) EU(0.04924)
Average: Cos(0.42090), NormCos(0.00394), EU(0.06515) for A photo of a tractor
First Last: Cos(0.42225), NormCos(0.00710), EU(0.09094) for A photo of a tractor
A photo of a semi-trailer truck
   0-to-1: Cosine(0.43051) NormCosine(0.00537) EU(0.07857)
   0-to-2: Cosine(0.43307) NormCosine(0.01074) EU(0.11107)
   1-to-2: Cosine(0.42848) NormCosine(0.00876) EU(0.10049)
Average: Cos(0.43069), NormCos(0.00829), EU(0.09671) for A photo of a s

In [ ]:
query_dict.keys()

In [ ]:
display_query_dict(query_dict)

In [ ]:
start=900
end=start+50
# query_name = 'A photo of a motorcycle'
# query_name = 'A photo of a frog'
query_name = 'frog'
display_top_query_from_meta(query_dict[query_name]['metadata'][start:end], fill=False, score_list=query_dict[query_name]['D'][start:end])

In [5]:
folder_path = "/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/"
# model_name = "ViT-B/32"
model_name = "RN50"


# feature_name = faiss_utils.get_feature_name(folder_path, model_name)
# normalize_features = load_pickle(feature_name)
# normalize_features.dtype == np.float32
k_near_faiss = KNearestFaiss(folder_path, model_name)

Loading from folder /scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/all_folders.pickle
Size of dataset is 2740000
Need to use brute force search without building an index


In [6]:

# image_path = "/compute/autobot-0-19/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_25/1/images/5411319415.jpg"
image_path = '/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/216/images/2375750164.jpg'
normalize_image_feature = k_near_faiss.get_normalized_image_feature(image_path)

# D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=3)


In [7]:
D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=3)

RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /__w/faiss-wheels/faiss-wheels/faiss/faiss/gpu/StandardGpuResources.cpp:410: Error: 'err == cudaSuccess' failed: Failed to cudaMalloc 1610612736 bytes on device 0 (error 2 out of memory
Outstanding allocations:


In [ ]:
ImageDisplay(image_path)

In [ ]:
display_top_query_from_meta(meta_list)
meta_list[0].get_path()

In [ ]:
import time
def show_top_query_images(query, start_idx=0, end_idx=40, fill=True):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_normalized_text_feature(query)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    display_top_query_from_meta(meta_list[start_idx:end_idx], fill=fill, score_list=score_list[start_idx:end_idx])

def grab_top_query_images(query, start_idx=0, end_idx=40):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_normalized_text_feature(query)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    print(f"length is {len(meta_list)} and {len(meta_list[start_idx:end_idx])}")
    return score_list[start_idx:end_idx], meta_list[start_idx:end_idx]

def grab_top_query_images_difference(query, diff_queries, start_idx=0, end_idx=40, lmb=0.25):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_text_difference_feature(query, diff_queries=diff_queries, lmb=lmb)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    print(f"length is {len(meta_list)} and {len(meta_list[start_idx:end_idx])}")
    return score_list[start_idx:end_idx], meta_list[start_idx:end_idx]
#     print(meta_lists[0][7].get_path())

In [ ]:

fruit_apple = 'a photo of apple, a type of fruit'
phone_apple = 'a photo of apple, a type of phone'
logo_apple = 'a photo of apple, a type of logo'
expensive_apple = 'a photo of expensive apple'
cheap_apple = 'a photo of cheap apple'
just_apple = 'a photo of apple'

just_an_apple = 'a photo of an apple'
just_apples = 'a photo of apples'
just_apple = 'a photo of apple'
apple_tree = 'a photo of apple tree'

no_apple = 'no apple'

old_apple = 'a photo of apple, which is old'
new_apple = 'a photo of apple, which is new'
fresh_apple = 'a photo of apple, which is fresh'
rotten_apple = 'rotten apple'
pretty_apple = 'a photo of apple, which is pretty'
ugly_apple = 'a photo of apple, which is ugly'
broken_apple = 'broken apple'
no_function_apple = 'not functioning apple'


old_car = "a photo of a car, which is old"
a1980_car = "a photo of a car in 1980s"
n2004_car = "a photo of a car in 2004"
n2005_car = "a photo of a car in 2005"
n2006_car = "a photo of a car in 2006"
n2007_car = "a photo of a car in 2007"
n2008_car = "a photo of a car in 2008"
n2009_car = "a photo of a car in 2009"
n2010_car = "a photo of a car in 2010"
n2011_car = "a photo of a car in 2011"
n2012_car = "a photo of a car in 2012"
n2013_car = "a photo of a car in 2013"
n2014_car = "a photo of a car in 2014"
a2010_car = "a photo of a car in 2010s"
new_car = "a photo of a car, which is new"
broken_car = "a photo of a car, which is broken"
just_car = "a photo of a car"

old_bike = "a photo of a bike, which is old"
new_bike = "a photo of a bike, which is new"
broken_bike = "a photo of a bike, which is broken"
just_bike = "a photo of a bike"

old_motorcycle = "a photo of a motorcycle, which is old"
new_motorcycle = "a photo of a motorcycle, which is new"
broken_motorcycle = "a photo of a motorcycle, which is broken"
just_motorcycle = "a photo of a motorcycle"
just_motorcycles = "a photo of motorcycles"

old_motorbike = "a photo of a motorbike, which is old"
new_motorbike = "a photo of a motorbike, which is new"
broken_motorbike = "a photo of a motorbike, which is broken"
just_motorbike = "a photo of a motorbike"
only_motorbike = "only motorbike"
motorbike_and_car = "motorbike and car"

no_car_motorbike = "a photo of a motorbike, no car"

old_jet = "a photo of a jet, which is old"
new_jet = "a photo of a jet, which is new"
broken_jet = "a photo of a jet, which is broken"

old_shirt = "a photo of a t-shirt, which is old"
new_shirt = "a photo of a t-shirt, which is new"

n2004 = 'a photo in 2004'
n2014 = 'a photo in 2014'
n1980 = 'a photo in 1980'
n2020 = 'a photo in 2020'


mouse_animal = 'a photo of mouse, a type of animal'
mouse_accessory = 'a photo of mouse, a type of accessory'
just_mouse = 'a photo of mouse'
just_star = 'a photo of star'

obama = 'obama'
trump = 'trump'
president = 'a photo of president'
curr_president = 'a photo of current president'
vice_president = 'a photo of vice president'
presidential_debate = "presidential debate"

In [ ]:
motor_bike_no_car_scores, motor_bike_no_car_lst = grab_top_query_images_difference("A photo of a motorcycle", diff_queries=['A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)


In [ ]:
start=500
end=start+50
display_top_query_from_meta(motor_bike_no_car_lst[start:end], fill=False, score_list=motor_bike_no_car_scores[start:end])
print(motor_bike_no_car_scores[end])
print(motor_bike_no_car_lst[8].get_path())

In [ ]:
motor_bike_with_car_scores, motor_bike_with_car_lst = grab_top_query_images_difference("A photo of a motorcycle", diff_queries=['A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=-1.)


In [ ]:
start=100
end=start+50
display_top_query_from_meta(motor_bike_with_car_lst[start:end], fill=False, score_list=motor_bike_with_car_scores[start:end])

In [ ]:
# show_top_query_images("A photo of motorbike without car", start_idx=0, end_idx=50, fill=False)
motor_bike_scores, motor_bike_lst = grab_top_query_images("A photo of a motorcycle", start_idx=0, end_idx=10000)

In [ ]:
plot_scores_jupyter(motor_bike_scores)

In [ ]:
def plot_scores_jupyter(score_list, plot_mean=False):
    min_count = None
    max_count = None
    avg_count = 0
    for s in score_list:
        if not min_count or s < min_count:
            min_count = s
        if not max_count or s > max_count:
            max_count = s
        avg_count += s
    avg_count = avg_count / len(score_list)

    bins = np.linspace(min_count, max_count, 15)
    plt.figure(figsize=(8,8))
    if plot_mean:
        plt.axhline(y=avg_count, label=f"Mean scores of Samples {avg_count}", linestyle='--', color='black')
    plt.hist(score_list, bins, alpha=0.5, label='Number of samples')
    plt.legend(loc='upper right')
    plt.tight_layout()

In [ ]:
start=100
end=start+50
display_top_query_from_meta(motor_bike_lst[start:end], fill=False, score_list=motor_bike_scores[start:end])
print(motor_bike_scores[end])
print(motor_bike_lst[8].get_path())

In [ ]:
motorcycle_with_car_path = "/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/33/images/3878798703.jpg"
motorcycle_without_car_path = motor_bike_lst[8].get_path()
ImageDisplay(motorcycle_with_car_path)


In [ ]:
normalize_image_feature = k_near_faiss.get_normalized_image_feature(motorcycle_with_car_path)
D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=20)

In [ ]:
idx = 10
display_top_query_from_meta(meta_list, fill=False)
p = meta_list[idx].get_path()
motor_score = k_near_faiss.get_clip_score(p, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(p, "A photo of a car")
print(f"motorcycle score: {motor_score}")
print(f"car score: {car_score}")
# ImageDisplay(p)

In [ ]:
motor_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, "A photo of a car")
print(f"motorcycle score: {motor_score:.4f}")
print(f"car score:        {car_score:.4f}")
lmb = 1.0
diff_clip_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, query="A photo of a motorcycle",\
                            diff_queries=["A photo of a car"],\
                           lmb=lmb)
print(f"Score difference: {diff_clip_score:.4f}")
ImageDisplay(motorcycle_with_car_path)

In [ ]:
motor_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, "A photo of a car")
print(f"motorcycle score: {motor_score:.4f}")
print(f"car score:        {car_score:.4f}")
lmb = 1.0
diff_clip_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, query="A photo of a motorcycle",\
                            diff_queries=["A photo of a car"],\
                           lmb=lmb)
print(f"Score difference: {diff_clip_score:.4f}")
ImageDisplay(motorcycle_without_car_path)

In [ ]:
apple_scores, apple_lst = grab_top_query_images("A photo of a apple", start_idx=0, end_idx=10000)

In [ ]:
# display_top_query_from_meta(apple_lst[:50], fill=False, score_list=apple_scores[:50])
start=300
end=start+50
display_top_query_from_meta(apple_lst[start:end], fill=False, score_list=apple_scores[start:end])
print(apple_scores[end])

In [ ]:
plot_scores_jupyter(apple_scores)

In [ ]:
bike_scores, bike_lst = grab_top_query_images("A photo of a bike", start_idx=0, end_idx=10000)

In [ ]:
start=600
end=start+50
display_top_query_from_meta(bike_lst[start:end], fill=False, score_list=bike_scores[start:end])
print(bike_scores[end])
# print(bike_lst[8].get_path())

In [ ]:
bike_no_motor_bike_scores, bike_no_motor_bike_lst = grab_top_query_images_difference("A photo of a bike", diff_queries=['A photo of a motorcycle'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)



In [ ]:
start=500
end=start+50
display_top_query_from_meta(bike_no_motor_bike_lst[start:end], fill=False, score_list=bike_no_motor_bike_scores[start:end])
print(bike_no_motor_bike_scores[end])

In [ ]:
bike_no_motor_bike_no_car_scores, bike_no_motor_bike_no_car_lst = grab_top_query_images_difference("A photo of a bike", \
                                                                                                   diff_queries=['A photo of a motorcycle', 'A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)




In [ ]:
start=600
end=start+50
display_top_query_from_meta(bike_no_motor_bike_no_car_lst[start:end], fill=False, score_list=bike_no_motor_bike_no_car_scores[start:end])
print(bike_no_motor_bike_no_car_scores[end])

In [ ]:
car_scores, car_lst = grab_top_query_images("A photo of car", start_idx=0, end_idx=4000)

In [ ]:
# meta_lst = grab_top_query_images(phone_apple, start_idx=0, end_idx=2049)
# plot_time_jupyter(meta_lst)

In [ ]:
lst = grab_top_query_images(fruit_apple, start_idx=0, end_idx=500)
plot_time_jupyter(lst)

In [ ]:
lst = grab_top_query_images(fruit_apple, start_idx=0, end_idx=5000)
plot_time_jupyter(lst)

In [ ]:
# ImageDisplay("/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_25/37/images/11052978605.png")